In [ ]:
# Import packages
import os
import re  # regular expressions
import warnings
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em

import rioxarray as rxr

from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

from sklearn.datasets import dump_svmlight_file

from glob import glob

warnings.simplefilter('ignore')

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'BD', 'BA_DATA'))

In [ ]:
vege_fs84 = glob('VegeData_16day/*h08v04*')

In [ ]:
vege_fs84

In [ ]:
vege_fs85 = glob('VegeData_16day/*h08v05*')

In [ ]:
vege_fs85

In [ ]:
vege_path = os.path.join("VegeData_16day", "MOD13A2.A2000049.h08v05.006.2015136104428.hdf")

In [ ]:
tem_path = os.path.join("TemData", "MOD11A2.A2000049.h08v05.006.2015058135046.hdf")

In [ ]:
with rio.open(vege_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta

In [ ]:
# Print all of the subdatasets in the data
with rio.open(vege_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)

In [ ]:
# Create empty list to append arrays (of band data)
all_bands = []

# Open the pre-fire HDF4 file
with rio.open(vege_path) as dataset:
    
    # Loop through each subdataset in HDF4 file
    for name in dataset.subdatasets:
        
        # Use regular expression to identify if subdataset has b0 in the name (the bands)
        if re.search("NDVI", name):
            
            # Open the band subdataset
            with rio.open(name) as subdataset:
                modis_meta = subdataset.profile
                
                # Read band data as a 2 dim arr and append to list
                all_bands.append(subdataset.read(1))

# Stack pre-fire reflectance bands
vege_modis = np.stack(all_bands)
vege_modis.shape

In [ ]:
with rio.open(tem_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta

In [ ]:
# Print all of the subdatasets in the data
with rio.open(tem_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)

In [ ]:
# Create empty list to append arrays (of band data)
all_bands = []

# Open the pre-fire HDF4 file
with rio.open(tem_path) as dataset:
    
    # Loop through each subdataset in HDF4 file
    for name in dataset.subdatasets:
        
        # Use regular expression to identify if subdataset has b0 in the name (the bands)
        if re.search("LST_Day", name):
            
            # Open the band subdataset
            with rio.open(name) as subdataset:
                modis_meta = subdataset.profile
                
                # Read band data as a 2 dim arr and append to list
                all_bands.append(subdataset.read(1))

# Stack pre-fire reflectance bands
tem_modis = np.stack(all_bands)
tem_modis.shape

# Mask function

Pick any location, divide the neighborhood into 10 regions of interest in various directions, each region is within 50 km, 36 degree and 50 km sector as a region. 

In [ ]:
def sector_mask(shape,centre,radius,angle_range):
    """
    Return a boolean mask for a circular sector. The start/stop angles in  
    `angle_range` should be given in clockwise order.
    """

    x,y = np.ogrid[:shape[0],:shape[1]]
    cx,cy = centre
    tmin,tmax = np.deg2rad(angle_range)

    # ensure stop angle > start angle
    if tmax < tmin:
            tmax += 2*np.pi

    # convert cartesian --> polar coordinates
    r2 = (x-cx)*(x-cx) + (y-cy)*(y-cy)
    theta = np.arctan2(x-cx,y-cy) - tmin

    # wrap angles between 0 and 2*pi
    theta %= (2*np.pi)

    # circular mask
    circmask = r2 <= radius*radius

    # angular mask
    anglemask = theta <= (tmax-tmin)

    return circmask*anglemask

In [ ]:
vege_modis[0]

In [ ]:
vege_modis.shape

In [ ]:
vege_matrix = vege_modis.reshape(1200,1200)

In [ ]:
vege_matrix.shape

In [ ]:
vege_matrix

In [ ]:
mask = sector_mask(vege_matrix.shape, (400, 600), 50, (0,36)) # need to consider the boundary cases

In [ ]:
from matplotlib import pyplot as pp

In [ ]:
pp.imshow(vege_matrix)
pp.show()

In [ ]:
test_matrix = vege_matrix.copy()
test_matrix[~mask] = -3000
pp.imshow(test_matrix)
pp.show()

In [ ]:
test_matrix.shape

In [ ]:
test_matrix[mask]

In [ ]:
test_matrix[mask].shape

In [ ]:
a = test_matrix[mask]

In [ ]:
a[:][a[:] == -3000] = -2000

In [ ]:
np.mean(a)

In [ ]:
a

In [ ]:
b.shape

In [ ]:
pp.imshow(vege_matrix)
pp.show()

# Process each 8-day hdf file

Within a region, exact a small set of features, and ask is there any fire observed in the region in the past 16 days. Consider the intensity of fire, number of pixels with fire (a pixel with fire for 3 days are considered as 3 fire), for a region in the past 16 days.

In [ ]:
def compute_feature_set(file, sds, fill_value, new_value, row, column, radius):
    """
    Return a feature set of a single pixel with row number r and column number c in the 1200 * 1200 map from FILE.
    Feature set is computed based on the average value in 10 regions in one 8-day time frame for the feature SDS from FILENAMES.
    Fill_value will be replaced by new_value in the feature matrix.
    
    """
    all_bands = []
    with rio.open(file) as dataset:
        for name in dataset.subdatasets:
            if re.search(sds, name):
                with rio.open(name) as subdataset:
                    modis_meta = subdataset.profile
                    all_bands.append(subdataset.read(1))
    vege_modis = np.stack(all_bands)
    vege_matrix = vege_modis[0].reshape(1200,1200)
    feature_set = []
    for i in range(10):
        mask = sector_mask(vege_matrix.shape, (row, column), radius, (i, i + 36))
        sector = vege_matrix[mask]
        sector[:][sector[:] == fill_value] = new_value
        mean_value = np.mean(sector)
        feature_set.append(mean_value)
    return feature_set

In [ ]:
aa = compute_feature_set(vege_fs85[0], "NDVI", -3000, -2000, 400, 600, 50)

In [ ]:
aa

In [ ]:
bb = compute_feature_set(vege_fs85[0], "NDVI", -3000, -2000, 100, 600, 50)

In [ ]:
bb